In [72]:
import numpy as np 
import json as json
import glob as glob
import zipfile as zipfile
import json as json
from nltk.tokenize import word_tokenize
import pandas as pd 
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import textblob

In [73]:
pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_rows', 1000)

In [74]:
df=pd.read_csv('clean_votes.csv',error_bad_lines=False)
country_code=pd.read_csv('country_codes_final.csv',error_bad_lines=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563090 entries, 0 to 563089
Data columns (total 6 columns):
country    563090 non-null object
symbol     563090 non-null object
year       563090 non-null int64
date       563090 non-null object
voted      563090 non-null object
title      563090 non-null object
dtypes: int64(1), object(5)
memory usage: 25.8+ MB


In [75]:
df['symbol'].nunique()

3524

In [76]:
def recoding(symbol, title):
    if symbol =='A/RES/49/35G':
        return'University of Jerusalem Al Quds'
    if symbol=="A/RES/52/65":
        return'Applicability of the Geneva Convention relative to the Protection of Civilian Persons in Time of War, of 12 August 1949, to the occupied Palestinian territory, including Jerusalem, and the other occupied Arab territories'
    else:
        return title
        
    
df['title']=df.apply(lambda x: recoding(x['symbol'],x['title']), axis=1)

In [77]:
data=df[['symbol','title']].drop_duplicates()

data2=df[['symbol','year','voted','country','date']]

In [78]:
# data=pd.merge(df2, df3, on=['symbol']);
# data.info()

In [79]:
data.head()

,symbol,title
0,A/RES/38/49,Information from Non-Self-Governing Territories transmitted under Article 73e of the Charter of the United Nations
151,A/RES/38/39I,Investments in South Africa
301,A/RES/38/39D,Sanctions against South Africa
451,A/RES/38/39J,Oil embargo against South Africa
601,A/RES/38/39E,Programme of work of the Special Committee against Apartheid


In [80]:
data['title']=data['title'].apply(lambda y: y.replace("Permanent sovereignty of the Palestinian people in the occupied Palestinian Territory, including Jerusalem, and of the Arab population in the occupied Syrian Golan over their natural resources resolution / adopted by the General Assembly,RECORDED,Yes"\
                                                                  ,"Permanent sovereignty of the Palestinian people in the occupied Palestinian Territory, including Jerusalem, and of the Arab population in the occupied Syrian Golan over their natural resources"))
data['title']=data['title'].apply(lambda y: y.replace("Bilateral nuclear-arms negotiations resolutio / adopted by the General Assembly,RECORDED,Yes"\
                                                                  ,"Bilateral nuclear-arms negotiations"))

data['title']=data['title'].apply(lambda y: y.replace("CHEMICAL AND BACTERIOLOGICAL (BIOLOGICAL) WEAPONS,A/RES/40/92C,RECORDED,Yes"\
                                                                  ,"Chemical and Bacteriological (biological) weapons"))

data['title']=data['title'].apply(lambda y: y.replace("PROHIBITION OF CHEMICAL AND BACTERIOLOGICAL WEAPONS,A/RES/40/92A,RECORDED,Yes"\
                                                                  ,"Prohibition of Chemical and Bacteriological Weapons"))

data['title']=data['title'].apply(lambda y: y.replace("CHEMICAL AND BACTERIOLOGICAL (BIOLOGICAL) WEAPONS,A/RES/41/58C,RECORDED,Yes"\
                                                                  ,"Chemical and Bacteriological (biological) weapons"))

data['title']=data['title'].apply(lambda y: y.replace("PROHIBITION OF CHEMICAL AND BACTERIOLOGICAL WEAPONS,A/RES/41/58B,RECORDED,Yes"\
                                                                  ,"Prohibition of Chemical and Bacteriological Weapons"))

data['title']=data['title'].apply(lambda y: y.replace("NUCLEAR CAPABILITY OF SOUTH AFRICA,A/RES/45/56B,RECORDED,Yes"\
                                                                  ,"Nuclear Capability of South Africa"))

data['title']=data['title'].apply(lambda y: y.replace("IMPLEMENTATION OF DECLARATION ON DENUCLEARIZATION OF AFRICA,A/RES/45/56A,RECORDED,Yes"\
                                                                  ,"Implementation of Declaration on Denuclearization of Africa"))


data['title']=data['title'].apply(lambda y: y.replace("Situation of human rights in the Democratic People,RECORDED,Yes"\
                                                                  ,"Situation of human rights in the Democratic People's Republic of Korea"))

data['title']=data['title'].apply(lambda y: y.replace("PREVENTION OF AN ARMS RACE,A/RES/45/55A,RECORDED,Yes"\
                                                                  ,"Prevention of an Arm Race"))

data['title']=data['title'].apply(lambda y: y.replace("CONFIDENCE-BUILDING MEASURES IN OUTER SPACE,A/RES/45/55B,RECORDED,Yes"\
                                                                  ,"Confidence-building measures in outer space"))


data['title']=data['title'].apply(lambda y: y.replace("QUESTION OF ANTARCTICA,A/RES/41/88C,ROLL-CALL,Yes"\
                                                                  ,"Question of Antarctica"))


data['title']=data['title'].apply(lambda y: y.replace("QUESTION OF ANTARCTICA,A/RES/41/88A,ROLL-CALL,Yes"\
                                                                  ,"Question of Antarctica"))


data['title']=data['title'].apply(lambda y: y.replace("QUESTION OF ANTARCTICA,A/RES/41/88B,ROLL-CALL,Yes"\
                                                                  ,"Question of Antarctica"))

data['title']=data['title'].apply(lambda y: y.replace("REDUCTION OF MILITARY BUDGETS,A/RES/40/91B,RECORDED,Yes"\
                                                                  ,"Reduction of Military Budgets"))

# data['title']=data['title'].apply(lambda y: y.replace("University of Jerusalem "Al Quds" resolution / adopted by the General Assembly,A/RES/49/35G,RECORDED,Yes"\
#                                                                   ,"University of Jerusalem Al Quds"))

data['title']=data['title'].apply(lambda y: y.replace("RECORDED - No machine generated vote,Yes"\
                                                                  ,"Prevention of an arms race in outer space"))

# data['title']=data['title'].apply(lambda y: y.replace("Applicability of the Geneva Convention relative to the Protection of Civilian Persons in Time of War, of 12 August 1949, to the occupied Palestinian territory, including Jerusalem, and the other occupied Arab territories resolution / adopted by the General Assembly,RECORDED,Yes"\
#                                                                   ,"Applicability of the Geneva Convention relative to the Protection of Civilian Persons in Time of War, of 12 August 1949, to the occupied Palestinian territory, including Jerusalem, and the other occupied Arab territories"))






In [81]:
def words_lower(text):
    words=[word.lower() for word in text.split()]
    return " ".join(words)

data['descr']=data['title'].map(words_lower)

In [82]:
data['descr']=data['descr'].apply(lambda y: y.translate(str.maketrans('','',string.punctuation)))

In [83]:
data['descr1']=data['descr'].apply(lambda x: str(x).strip())
data['descr1']=data['descr'].apply(lambda x: str(x).replace(" ",""))

In [84]:
# data1=data.groupby(['descr1','resolut','descr'])['resolut'].nunique().to_frame('Count')
# data1.reset_index(inplace=True)
# data2=data1.sort_values('Count', ascending=False)

In [85]:
# data2

In [86]:
# data3=data.groupby(['descr1'])['resolut'].unique().to_frame('Count')
# data3.reset_index(inplace=True)
# data4=data3.sort_values('Count', ascending=False)

In [87]:
# data2['descr']

In [88]:
def res_category(words):
#         if "crime" in words:
#             return "Test"
        if 'terrorism' in words:
            return "Combating Global Terrorism"
        if "women" in words:
            return "Women Advancement & Protection"
        if "crime" in words and "prevention" in words:
            return "Crime Prevention"
        if "technology" in words or "science" in words or "scientific and technological" in words:
            return "Science and Technology"
        
        if 'arms' in words and "race" in words or "outer space" in words:
            return "Arm Race in Outer Space & Water"
        if 'arm' in words and "race" in words:
            return "Arm Race in Outer Space & Water"
               
        if "investigate israeli practices" in words:
            return "Peace in Middle East"
        if "isareli practices affecting the human rights" in words:
            return "Peace in Middle East"
        if "alquds" in words:
            return "Peace in Middle East"
        if "occupied arab territories" in words:
            return "Peace in Middle East"
 
        
        if "permanent sovereignty of the palestinian" in words:
            return "Peace in Middle East"
        
        if 'syrian golan' in words: 
            return "Peace in Middle East"
        
        if 'jerusalem' in words:
            return 'Peace in Middle East'
        
        if 'middle east' in words:
            return "Peace in Middle East" #can be Israeli Occupation
        
        if 'palestine' in words or "palestinian" in words or "unrwa" in words\
        or '1967' in words and "refugees" in words: 
            return 'Peace in Middle East'
        
        if "1967 and subsequent hostilities" in words:
            return 'Peace in Middle East'
        
                
        if "arms" in words and "trade" in words and 'treaty' in words:
            return "Control of Small Arms"
        if "international arms transfers" in words:
            return "Control of Small Arms"
        if "small arms" in words:
            return "Control of Small Arms" 
        if "conventional arms" in words:
            return "Control of Small Arms" 
        if "conventional ammunition stockpiles in surplus" in words:
            return "Control of Small Arms"
        
        
        

        
        if 'nuclear' in words or 'proliferation' in words or\
        'disarmament' in words or "arms control" in words or\
        "atomic energy" in words or "missiles" in words or "weapons" in words:
            return 'Nuclear Disarmament'
        if "use, stockpiling, production" in words or "stockpiles" in words:
            return "Nuclear Disarmament"
        if "disarmament" in words or "armaments" in words:
            return 'Nuclear Disarmament'
        if "prohibition of the use stockpiling production" in words:
            return "Nuclear Disarmament"
        if "fissionable material" in words:
            return "Nuclear Disarmament"
        if "antiballistic missile" in words:
            return "Nuclear Disarmament"
        if "military capabilities" in words:
            return "Nuclear Disarmament" 
        if "military research and development" in words:
            return "Nuclear Disarmament" 
        if "objective information on military matters" in words:
            return "Nuclear Disarmament"  
            
            
        
        if "south" in words and "africa" in words:
            return "South Africa"
        if "southern" in words and "africa" in words:
            return "South Africa"
        
        if "in promoting development in the context of globalization and interdependence" in words:
            return "International Cooperation"

        if 'decolonization' in words or "colonial" in words\
        or "non-self-governing" in words or "self-governing" in words or "cultural property"\
        in words or "national sovereignty" in words or  "globalization and interdependence"\
        in words or "non-self governing territories" in words: 
            return 'Decolonization'
        if "nonselfgoverning territories" in words or "non selfgoverning territories" in words:
            return "Decolonization"
        if "governing territories" in words:
            return "Decolonization"
            
        
#         if 'arms' in words and "race" in words or "outer space" in words\
#         or "military research and development" in words or "measures to provide objective information\
#         on military capabilities" in words: 
#             return "Arm Race in Space"

        if 'oceans' in words and 'law' in words and 'sea' in words:
            return "Oceans & Law of the Sea"
        if "antarctica" in words:
            return "Oceans & Law of the Sea"
        if 'law' in words and 'sea' in words:
            return "Oceans & Law of the Sea"
        
        #countries
        
        if "kuwait" in words:
            return "Kuwait"
        if "central african republic" in words:
            return "Central African Republic"
        
        if "iran" in words:
            return "Iran"
        if "cuba" in words:
            return "Cuba"
        if "lebanon" in words:
            return "Lebanon"
        if "namibia" in words:
            return 'Namibia'
        if "kampuchea" in words:
            return 'Kampuchea'
        if "chile" in words:
            return "Chile"
        if "mayotte" in words:
            return "Mayotte"

        if "iraq" in words:
            return "Iraq"
        if "korea" in words:
            return "North Korea"
        if "sudan" in words:
            return "Sudan"
        if 'syrian' in words or "syria" in words:
            return "Syria"
        if "congo" in words:
            return "Congo"
        if "salvador" in words:
            return "Salvador"
        if "myanmar" in words:
            return "Myanmar"
        if "guatemala" in words:
            return "Guatemala"
        if "kosovo" in words or "yugoslavia" in words or "herzegovina" in words\
        or "croatia" in words or "montenegro" in words or "serbia" in words:
            return "Yugoslavia"
        if "afghanistan" in words:
            return "Afghanistan"
        if "turkmenistan" in words:
            return "Turkmenistan"
        if "nigeria" in words:
            return "Nigeria"
        if "belarus" in words:
            return "Belarus"
#         if "uzbekistan" in words:
#             return "Uzbekistan"
#         if "kuwait" in words:
#             return "Kuwait"
#         if "bolivia" in words:
#             return "Bolivia"
        if "ukraine" in words or "crimea" in words:
            return "Ukraine"
        if "angola" in words:
            return "Angola"
        if "malvinas" in words:
            return "Malvinas"
#         if "uzbekistan" in words:
#             return "Uzbekistan"
#         if "bolivia" in words:
#             return 'Bolivia'
        if "libya" in words:
            return "Libya"
        if "nicaragua" in words:
            return "Nicaragua"
        if "ethiopia" in words:
            return "Ethiopia"
#         if "mozambique" in words:
#             return "Mozambique"
#         if "uganda" in words:
#             return "Uganda"
#         if "chad" in words:
#             return "Chad"
#         if "maldives" in words:
#             return "Maldives"
        if "central america" in words:
            return "Central America"    
        if 'georgia' in words:
            return "Georgia"
        if "africa" in words:
            return "Africa"
        if "virgin islands" in words:
            return "Virgin Islands"
        if "guam" in words:
            return "Guam"
        if "east timor" in words:
            return "East Timor"

        

        if "child" in words or "children" in words:
            return "Rights of the Child"

        
        if "apartheid" in words or "racial discrimination" in words or "discrimination" in words or\
           "nazi" in words or "racial intolerance" in words:
            return "Elimination of Racial Discrimination"
        
        if "death penalty" in words:
            return "Abolishing Death Penalty"

        if "human" in words and "rights" in words or "human rights" in words or "the right to development" in words or\
        "the right to food" in words or "political rights" in words or "rights of the child" in words\
        or "right to development" in words or "right of peoples to peace" in words\
        or "preparation of societies for life in peace" in words or "realization of the right to adequate housing"\
        in words or "improvement of social life" in words or "the right of peoples to self-determination" in words\
        or "the right to peace" in words or "economic rights" in words or "right to universal freedom of travel" in\
        words or "the human right to water and sanitation" in words or "the right of development" in words or\
        "the right of everyone" in words or "respect for the universal freedom" in words\
        or "social and economic changes for the purpose of social progress" in words: 
            return "Universal Human Rights" 
        if "the right of peoples to selfdetermination" in words:
            return "Universal Human Rights" 
        if "the right of universal freedom" in words:
            return "Universal Human Rights" 
        if "rights of indigenous peoples" in words:
            return "Universal Human Rights" 
        if "the rights and dignity of older persons" in words:
            return "Universal Human Rights" 
        if "the rights of persons with disabilities" in words:
            return "Universal Human Rights" 
            
            
            
            

        
        if "protection of migrants" in words or "international cooperation to avert new flows of refugees" in words\
        or "united nations high commissioner for refugees" in words:
            return "Protection of Migrants & Refugees"
        
        if "security council" in words:
            return "Other"
        if "agriculture development" in words:
            return "Other"
        if "security and safety of diplomatic" in words:
            return "Other"
        if "interorganizational security measures" in words:
            return "Other"
        
        if "security" in words:
            return "International Security"
        
        if "oil slick on lebanese shores" in words or "environment" in words\
        or "dumping of radioactive wastes" in words or "modern energy for all" in words\
        or "sustainability and development" in words or "sustainable" in words or "nature" in words:
            return "Support for Clean Environment"
        if "uses of international watercourses" in words:
            return "Support for Clean Environment"
        if "global climate" in words:
            return "Support for Clean Environment"
            
            
        
        if "promotion of democratization" in words or "promoting and consolidating democracy"\
        in words or "democratization" in words or "principle of periodic and genuine elections" in words:
            return "Democratization"
        

        if "trade" in words or 'commodities' in words:
            return "Trade and Commodities"

        
        
        if "embargo" in words:
            return "Embargo"
        
        
        if "conversion of military resources" in words or "military budgets" in\
        words or "reduction of military budgets" in words:
            return "Military Resources Reduction"

        
        if "peace" in words:
            return "Support for Peacemaking"
        

        
        if "international economic order" in words:
            return "Economic Order & Cooperation"
        if "debt restructuring processes" in words:
            return "Economic Order & Cooperation"
        if "international financial system and development" in words:
            return "Economic Order & Cooperation"
        if "debt crisis and development" in words:
            return "Economic Order & Cooperation"
        if "development and international economic cooperation" in words:
            return "Economic Order & Cooperation"
            
        
        if "developing countries" in words or "developing to developed countries" in words or\
        "coercive" in words and "measures" in words:
            return "Support to Developing Countries"
        if "official development assistance" in words:
            return "Support to Developing Countries"
        
        if "religious" in words or "religion" in words:
            return "Eliminating Religious Intolerance & Islamophobia"
        
        if "external debt problems" in words:
            return "Economic Order & Cooperation"

        
        if "funding" in words or "fund" in words or "financial" in words or "budget" in words or "financing" in words\
        or "income estimates" in words or "expenses" in words or "finance" in words or "reimbursement" in words\
        or "debt" in words:
            return "UN Funding & Budget"
        
        if "united nations" in words and "strengthening" in \
        words or "strengthening of good-neighbourliness between states" in words:
            return "UN Strengthening"
        
        if "arab states" in words:
            return "League of Arab States"


        if "cooperation" in words:
            return "International Cooperation"
        
        if "questions relating to information" in words or "question relating to information" in words\
        or 'questions related to information' in words:
            return "Freedom of Information"
        
        if "special economic assistance" in words or "special action" in words or "special assistance" in words\
        or "emergency assistance" in words or "assistance" in words:
            return "Assistance to Frontline States"
        
        if "arbitrary executions" in words:
            return "Arbitrary Executions"
        
        if "brokering activities" in words:
            return "Preventing Brokering Activities"
        
        if "host country" in words:
            return "Relations with Host Country"
        
        if 'measures' in words and "authority" in words and "geneva" in words:
            return "1925 geneva protocol"
        if "1925 geneva" in words:
            return "1925 geneva protocol"
            
        
        if "neighbourliness between states" in words:
            return "Neighborliness between States"
        

        
#         if "implenmentation" in words or "implementations" in words or "implenmentation of the declaration"\
#         in words or "implementations of the recommendations" in words:
#             return "Implenmentation"
        
        if "international affairs" in words or "international situation" in words or "international relations" in words:
            return "International Relations"
        if "confidencebuilding in international economic relations" in words:
            return "International Relations"
        if "confidencebuilding measures" in words:
            return "International Relations"
            
        
        if "world social situation" in words:
            return "Eliminating World Inequality"
        if "staff" in words or "honoraria payable" in words or "pension system" in words:
            return "Admin"
        if "pension scheme" in words:
            return "Admin"
        if "health insurance contributions" in words:
            return "Admin"

            
        if "promotion of a democratic and equitable international order" in words:
            return "International Order"
        if "special session" in words:
            return "Special Sessions"
        if "western sahara" in words:
            return "Western Sahara"
        if "remnants of war" in words:
            return "Removing Remnants of War"
        else:
            return 'Other'
        
data['category']=data['descr'].map(res_category);

In [89]:
data['category'].value_counts()

          

Peace in Middle East                                840
Nuclear Disarmament                                 774
Universal Human Rights                              231
Decolonization                                      221
Other                                               123
South Africa                                        102
Elimination of Racial Discrimination                86 
UN Funding & Budget                                 72 
Arm Race in Outer Space & Water                     69 
Oceans & Law of the Sea                             56 
Special Sessions                                    56 
Science and Technology                              53 
International Security                              50 
Control of Small Arms                               48 
Namibia                                             43 
Support for Peacemaking                             40 
Support to Developing Countries                     39 
Lebanon                                         

In [90]:
df3=data[data['category']=='Death Penalty']
df3

,symbol,title,descr,descr1,category


In [91]:
data['category'].nunique()

78

In [92]:
df1=data[['symbol','title','descr1','category']]

df1['category'].unique()

array(['Decolonization', 'South Africa',
       'Elimination of Racial Discrimination', 'Nuclear Disarmament',
       'Peace in Middle East', 'Oceans & Law of the Sea',
       'Arm Race in Outer Space & Water', 'Freedom of Information',
       'Support for Peacemaking', 'Other', 'Guatemala',
       'International Security', 'Economic Order & Cooperation',
       'International Relations', 'Women Advancement & Protection',
       'Universal Human Rights', 'Science and Technology', 'Salvador',
       'Chile', 'Africa', 'Support to Developing Countries',
       'Removing Remnants of War', 'Special Sessions',
       'Military Resources Reduction', 'UN Strengthening',
       'UN Funding & Budget', 'Admin', 'Kampuchea',
       'League of Arab States', 'Malvinas', 'Afghanistan', 'Mayotte',
       'Western Sahara', 'Namibia', 'Lebanon', 'Abolishing Death Penalty',
       'Support for Clean Environment', 'Combating Global Terrorism',
       'International Cooperation', 'Eliminating World Inequa

In [94]:
merge1=pd.merge(df1, data2, on=['symbol']);

merge1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 563090 entries, 0 to 563089
Data columns (total 8 columns):
symbol      563090 non-null object
title       563090 non-null object
descr1      563090 non-null object
category    563090 non-null object
year        563090 non-null int64
voted       563090 non-null object
country     563090 non-null object
date        563090 non-null object
dtypes: int64(1), object(7)
memory usage: 38.7+ MB


In [95]:
merge2=pd.merge(merge1,country_code, on=['country']);
merge2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551784 entries, 0 to 551783
Data columns (total 10 columns):
symbol      551784 non-null object
title       551784 non-null object
descr1      551784 non-null object
category    551784 non-null object
year        551784 non-null int64
voted       551784 non-null object
country     551784 non-null object
date        551784 non-null object
a2          551784 non-null object
a3          551784 non-null object
dtypes: int64(1), object(9)
memory usage: 46.3+ MB


In [96]:
def coded_voted(voted):
    for x in voted:
        if voted=='Y':
            return 1
        if voted=='N':
            return -1
        if voted =='A':
            return 0
        
merge2['coded_vote']=merge2['voted'].map(coded_voted)

In [97]:
df_greg=merge2[['symbol','title','category','year','date','country','a3','voted','coded_vote']]

df_greg

,symbol,title,category,year,date,country,a3,voted,coded_vote
0,A/RES/38/49,Information from Non-Self-Governing Territories transmitted under Article 73e of the Charter of the United Nations,Decolonization,1983,12/7/1983,AFGHANISTAN,AFG,Y,1
1,A/RES/38/39I,Investments in South Africa,South Africa,1983,12/5/1983,AFGHANISTAN,AFG,Y,1
2,A/RES/38/39D,Sanctions against South Africa,South Africa,1983,12/5/1983,AFGHANISTAN,AFG,Y,1
3,A/RES/38/39J,Oil embargo against South Africa,South Africa,1983,12/5/1983,AFGHANISTAN,AFG,Y,1
4,A/RES/38/39E,Programme of work of the Special Committee against Apartheid,Elimination of Racial Discrimination,1983,12/5/1983,AFGHANISTAN,AFG,Y,1
5,A/RES/38/39G,Military and nuclear collaboration with South Africa,Nuclear Disarmament,1983,12/5/1983,AFGHANISTAN,AFG,Y,1
6,A/RES/38/39K,Apartheid in Sports,Elimination of Racial Discrimination,1983,12/5/1983,AFGHANISTAN,AFG,Y,1
7,A/RES/38/39F,Relations between Israel and South Africa,South Africa,1983,12/5/1983,AFGHANISTAN,AFG,Y,1
8,A/RES/38/39C,Effects of apartheid on the countries of southern Africa,South Africa,1983,12/5/1983,AFGHANISTAN,AFG,Y,1
9,A/RES/38/39B,Programme of Action against Apartheid,Elimination of Racial Discrimination,1983,12/5/1983,AFGHANISTAN,AFG,Y,1


In [99]:
df_greg.to_csv(path_or_buf="/Users/uncontract/Documents/UN_data/UN_Resolution/PART3/data_frame_greg1.csv", index=False)

In [22]:
merge['descr1'].nunique()

# merge.info()

930

In [461]:
merge.to_csv(path_or_buf="/Users/uncontract/Documents/UN_data/UN_Resolution/PART3/categories_titles.csv", index=False)

In [462]:
merge['category'].nunique()

78